Θυμήσου. όταν τρέχει κάποιο script το οποίο κάνει input ένα αρχειο από το σκληρό. θα πρέπει αυτό το αρχείο να βρίσκεται στο path του αρχικού script.


In [2]:
import sys

sys.path.append('..')
#sys.path.append('../preprocessing')

import importcsv

print('Importing csv file...........')
small_dataset = importcsv.opencsv('../processedspeech2.csv') 

Importing csv file...........


In [3]:
#Import the json file with the inverted catalog
import json

file_path = "inverted_catalog2.json"

# Load the dictionary from the file
with open(file_path, 'r') as file:
    loaded_dictionary = json.load(file)

In [5]:
# the querry
sys.path.append('../preprocessing')
from wordtools import process_string

querry = 'σκανδαλα βατοπεδίου'

querry = process_string(querry)

print(querry)

ΣΚΑΝΔΑΛ ΒΑΤΟΠΕΔΙ


In [6]:
#split the querry into words and check if they are indexed in the inverted catalog. return the indexes of the speeches that contain the querry words
querrylist = querry.split(' ')
indices = []
for word in querrylist:
    if word in loaded_dictionary.keys() and word not in indices:
        if loaded_dictionary[word] not in indices:
            indices = indices + loaded_dictionary[word]
    else:
        print('Maybe use the deep search')
indices 

[28,
 33,
 43,
 44,
 69,
 100,
 112,
 130,
 141,
 145,
 147,
 168,
 169,
 171,
 172,
 190,
 191,
 192,
 205,
 215,
 218,
 230,
 236,
 242,
 244,
 250,
 256,
 260,
 261,
 268,
 269,
 270,
 275,
 277,
 279,
 287,
 294,
 302,
 323,
 324,
 326,
 327,
 328,
 339,
 340,
 342,
 349,
 351,
 352,
 392,
 425,
 430,
 435,
 442,
 443,
 453,
 459,
 463,
 465,
 471,
 491,
 500,
 501,
 502,
 512,
 521,
 522,
 523,
 526,
 527,
 531,
 533,
 535,
 536,
 552,
 554,
 555,
 573,
 589,
 591,
 603,
 621,
 623,
 624,
 625,
 641,
 689,
 746,
 762,
 763,
 861,
 938,
 951,
 952,
 953,
 959,
 961,
 963,
 973,
 985,
 1013,
 1017,
 1023,
 1060,
 1605,
 1687,
 1697,
 1766,
 1807,
 1809,
 1866,
 1870,
 1873,
 1875,
 1876,
 1886,
 1887,
 1896,
 1908,
 1913,
 1914,
 1919,
 1924,
 1935,
 1942,
 1957,
 1965,
 1973,
 2080,
 2092,
 2109,
 2119,
 2251,
 2330,
 2334,
 2336,
 2341,
 2353,
 2355,
 2366,
 2372,
 2379,
 2384,
 2399,
 2424,
 2448,
 2450,
 2456,
 2457,
 2462,
 2471,
 2480,
 2481,
 2499,
 2501,
 2503,
 2509,
 2521,

In [8]:
# remove duplicate indices
indices = list(set(indices))

In [9]:
#make a list from the indexes 
smaller_list = small_dataset['processed_speeches'].iloc[indices].to_list()


In [10]:
# Add as last row the querr. (Small note: the inverted catalog has the words in lower but the processed speeches are in upper)
smaller_list.append(querry)


In [11]:
# tfid vectorizer as usual 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(smaller_list)

In [12]:
#cosine similarity
ddsim_matrix = cosine_similarity(tfidf_matrix[len(smaller_list)-1,:], tfidf_matrix)

In [13]:
print(ddsim_matrix)

[[0.0176286  0.00404974 0.00335973 ... 0.01010271 0.00350536 1.        ]]


In [14]:
#remove assosiations less than 0
similar = {}
for i in range(len(smaller_list)-1):
    if ddsim_matrix[0,i] > 0:
        similar[i] = ddsim_matrix[0,i]

In [15]:
similar

{0: 0.017628601359071516,
 1: 0.004049743947965205,
 2: 0.003359727246955328,
 3: 0.002616449512929516,
 4: 0.017695917853162126,
 5: 0.014917271141664466,
 6: 0.004175547457464689,
 7: 0.00498440297186282,
 8: 0.0013213623745298721,
 9: 0.002485805843239885,
 10: 0.0028215858445351373,
 11: 0.0044248627217641796,
 12: 0.0032308809630836248,
 13: 0.0032153610486903555,
 14: 0.006816951900469526,
 15: 0.01306673054115758,
 16: 0.0026804079837283554,
 17: 0.0020318478690134188,
 18: 0.0024930251719628657,
 19: 0.0036642400093624974,
 20: 0.015684057533862365,
 21: 0.0026865430608691613,
 22: 0.0023242937830220277,
 23: 0.010869174520120173,
 24: 0.002304566356060943,
 25: 0.011206598212236518,
 26: 0.007297973100078652,
 27: 0.0033130384088098466,
 28: 0.0032295822467726374,
 29: 0.006348364589895915,
 30: 0.020703391493016483,
 31: 0.04285270623499425,
 32: 0.005954313589238028,
 33: 0.007164198493610183,
 34: 0.004708826326417654,
 35: 0.0024263860845135086,
 36: 0.0036336470446918898,

In [16]:
#Find the top k similarity amongst the speeches based on querry
from heapq import nlargest

k = 80
top_k_items = nlargest(k, similar.items(), key=lambda x: x[1])

In [17]:
#print the top 5 speeches related to the querry

for i,list in enumerate(top_k_items):    
    print('Similar speech:', i)
    print('speech:',small_dataset.at[indices[list[0]],'speech'])
    #print(dataset3.at[speechstr,'processed1'])
    print('member_name:',small_dataset.at[indices[list[0]],'member_name'])
    print('political_party:',small_dataset.at[indices[list[0]],'political_party'])
    print('sitting_date:',small_dataset.at[indices[list[0]],'sitting_date'])
    if i == 6:
        break

Similar speech: 0
speech:  Ευχαριστούμε τον κ. Κουβέλη και σας ευχαριστώ και εγώ προσωπικά, κύριε συνάδελφε, που δεν βάλατε καθόλου Βατοπέδιο, γιατί εδώ κινδυνεύουμε όλοι να γίνουμε Βατοπέδιο και να μιλάμε για το Βατοπέδιο και όχι για τα νομοσχέδια!.
member_name: σουρλας ιωαννη γεωργιος
political_party: νεα δημοκρατια
sitting_date: 15/10/2008
Similar speech: 1
speech:  Τελειώνω, κύριε Πρόεδρε. Το σκάνδαλο της Μονής Βατοπεδίου είναι ένα πολλαπλό σκάνδαλο. Είναι το σκάνδαλο της παραίτησης από τα δικαίωμα του δημοσίου, είναι το σκάνδαλο της ανταλλαγής, είναι το σκάνδαλο της συγκάλυψης, είναι το σκάνδαλο της συνεχιζόμενης εγκατάλειψης των ακινήτων και των δικαιωμάτων του δημοσίου, γιατί η Κυβέρνηση δεν προβαίνει σε ενέργειες υπεράσπισης των συμφερόντων και των δικαιωμάτων του δημοσίου, παρ’ ό,τι την έχουμε καλέσει να το κάνει και αρνείται να το κάνει και αυτό συντελείται στην τρέχουσα βουλευτική περίοδο και όχι στην περασμένη. Επίσης το σκάνδαλο της Μονής Βατοπεδίου είναι ένα τεράστιο χρημ